In [1]:
console.log(Deno.systemMemoryInfo());
console.log(Deno.loadavg());

[Object: null prototype] {
  total: 17114173440,
  free: 8160628736,
  available: 0,
  buffers: 0,
  cached: 0,
  swapTotal: 2550136,
  swapFree: 0
}
[ 0, 0, 0 ]
